In [31]:
import torch
from onnx_tf.backend import prepare
import onnx
import tensorflow as tf

In [32]:
loaded = torch.jit.load('7x7/best.pt')
# loaded = torch.jit.load('5x5/best.pt')
# loaded = torch.jit.load('src/dualnet5x5_v2.pt')
# loaded = torch.jit.load('src/dualnet5x5_se.pt')
# loaded = torch.jit.load('src/dualnet7x7_se.pt')
print(loaded)

RecursiveScriptModule(
  original_name=DualNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (downsample): RecursiveScriptModule(original_name=Sequential)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_nam

In [33]:
loaded.eval()
bisze = 7
dummy_input = torch.randn((1, 12, bisze, bisze))
torch.onnx.export(loaded, dummy_input, "dualnet.onnx", example_outputs=torch.rand((1, bisze*bisze+2)), verbose=True)

graph(%x.1 : Float(3, 12, 7, 7, strides=[588, 49, 7, 1], requires_grad=0, device=cpu),
      %pi.4.bias : Float(50, strides=[1], requires_grad=0, device=cuda:0),
      %pi.4.weight : Float(50, 1568, strides=[1568, 1], requires_grad=0, device=cuda:0),
      %v.4.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %v.4.weight : Float(32, 147, strides=[147, 1], requires_grad=0, device=cuda:0),
      %v.6.bias : Float(1, strides=[1], requires_grad=0, device=cuda:0),
      %v.6.weight : Float(1, 32, strides=[32, 1], requires_grad=0, device=cuda:0),
      %95 : Float(32, 12, 3, 3, strides=[108, 9, 3, 1], requires_grad=0, device=cuda:0),
      %96 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %98 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %99 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %101 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %102 : Float(32, stride

In [34]:
tf.autograph.experimental.do_not_convert
onnx_model = onnx.load('dualnet.onnx')

tf_rep = prepare(onnx_model, device='gpu')
tf_rep.export_graph('model.pb')

INFO:tensorflow:Assets written to: model.pb\assets


In [35]:
!cd train
!saved_model_cli show --dir ./model.pb --all

�w�肳�ꂽ�p�X��������܂���B



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x.1'] tensor_info:
        dtype: DT_FLOAT
        shape: (3, 12, 7, 7)
        name: serving_default_x.1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (3, 51)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
        Named Argument #1
          x.1

  Function Name: 'gen_tensor_dict'


In [36]:

!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' --saved_model_tags=serve model.pb js_model --quantize_uint8

Writing weight file js_model\model.json...
weight PartitionedCall/onnx_tf_prefix_Constant_0 with shape (4,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/Where with shape (0, 1) and dtype int64 was auto converted to the type int32
weight PartitionedCall/Cast with shape (1,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/GatherV2 with shape (0,) and dtype int64 was auto converted to the type int32
weight PartitionedCall/zeros with shape () and dtype int64 was auto converted to the type int32


2022-01-24 17:13:07.339591: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-24 17:13:07.499463: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-01-24 17:13:07.499944: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-01-24 17:13:07.587312: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:816] Optimization results for grappler item: graph_to_optimize
2022-01-24 17:13:07.587503: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:818]   function_optimizer: Graph size after: 278 nodes (251), 267 edges (241), time = 8.646ms.
2022-01-24 17:13:07.5